<a href="https://colab.research.google.com/github/solomontessema/Generative-AI-with-Python/blob/main/notebooks/Introduction_to_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>Day 12: RAG (Retrieval-Augmented Generation)</h1></td>
  </tr>
</table>

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
API_KEY = os.getenv("GPT_API_KEY")
client = OpenAI(api_key=API_KEY)

# Load knowledge base (simple text chunks for demo)
def load_knowledge_base():
    with open("knowledge_base.txt", "r", encoding="utf-8") as f:
        chunks = f.read().split("\n\n")  # Assume chunks are separated by double newlines
        #print(chunks)
    return chunks


def chat_with_rag():
    print("Welcome to IonnovaBot! Type 'exit' to quit.\n")
    messages = [{"role": "system", "content": "You are a helpful assistant that uses external knowledge to answer questions."}]
    kb_chunks = load_knowledge_base()

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("IonnovaBot: Goodbye!")
            break

        # Retrieve context

        context = "\n---\n".join(kb_chunks)

        # Inject context into prompt
        messages.append({"role": "user", "content": f"Context:\n{context}\n\nQuestion: {user_input}"})

        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0.7
        )

        reply = response.choices[0].message.content
        print(f"IonnovaBot: {reply}")
        messages.append({"role": "assistant", "content": reply})

chat_with_rag()


Welcome to IonnovaBot! Type 'exit' to quit.

You: who are ionnova?
IonnovaBot: IONNOVA is a technology company based in Frisco, Texas. They specialize in software engineering, AI automation, and data analytics. They design and develop custom web, mobile, and enterprise applications for businesses, focusing on automation, scalability, and performance optimization. Their work also involves building intelligent systems using machine learning, deep learning, and natural language processing for predictive analytics, decision-making, and business process automation. In addition, they offer advanced data analytics, AutoCAD customization, professional CAD drafting, direct placement services for IT professionals, and training in software development, data analytics, and AI. Their mission is to empower businesses through innovative software, intelligent automation, and data-driven strategies.


In [4]:
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("GPT_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "ionnova-rag"
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
index = pc.Index(index_name)

# Load and embed knowledge base
def load_and_embed_kb():
    with open("knowledge_base.txt", "r", encoding="utf-8") as f:
        chunks = f.read().split("\n\n")
    for i, chunk in enumerate(chunks):
        embedding = client.embeddings.create(
            model="text-embedding-ada-002",
            input=chunk
        ).data[0].embedding
        index.upsert([(f"chunk-{i}", embedding, {"text": chunk})])

# Retrieve top-k relevant chunks
def retrieve_context(query, top_k=5):
    query_embedding = client.embeddings.create(
        model="text-embedding-ada-002",
        input=query
    ).data[0].embedding
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    return [match["metadata"]["text"] for match in results["matches"]]

# Chat loop
def chat_with_rag():
    print("Welcome to IonnovaBot! Type 'exit' to quit.\n")
    messages = [{"role": "system", "content": "You are a helpful assistant that uses external knowledge to answer questions."}]

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("IonnovaBot: Goodbye!")
            break

        context_chunks = retrieve_context(user_input)
        context = "\n---\n".join(context_chunks)

        messages.append({"role": "user", "content": f"Context:\n{context}\n\nQuestion: {user_input}"})

        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0.7
        )

        reply = response.choices[0].message.content
        print(f"IonnovaBot: {reply}")
        messages.append({"role": "assistant", "content": reply})

# Run once to populate Pinecone
# load_and_embed_kb()

chat_with_rag()


Welcome to IonnovaBot! Type 'exit' to quit.

You: who are ionnova?
IonnovaBot: I'm sorry, but I couldn't find any specific or detailed information about "ionnova." It's possible that there might be a spelling mistake or the entity isn't well-known or doesn't exist. If you have additional details or context, I'd be happy to try and provide a better answer.
You: What is IONNOVA?
IonnovaBot: I'm sorry, but I still couldn't find any specific or detailed information about "IONNOVA." It's possible that there might be a spelling mistake or the entity isn't widely recognized or doesn't exist. Please provide more details or context and I'll do my best to provide a helpful answer.


KeyboardInterrupt: Interrupted by user